# Case Study #1 - Danny's Diner

## Introduction
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

## Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:
- sales
- menu
- members

You can inspect the entity relationship diagram and example data below.

## Entity Relationship Diagram

<p align="center">
<img src="..\Images/erd 8weeks case study 1.png" width="400">
</p>

## Example Datasets
All datasets exist within the dannys_diner database schema - be sure to include this reference within your SQL scripts as you start exploring the data and answering the case study questions.

**Table 1: sales**

The sales table captures all customer_id level purchases with an corresponding order_date and product_id information for when and what menu items were ordered.

**Table 2: menu**

The menu table maps the product_id to the actual product_name and price of each menu item.

**Table 3: members**

The final members table captures the join_date when a customer_id joined the beta version of the Danny’s Diner loyalty program.

## Interactive SQL Session
You can use the embedded DB Fiddle below to easily access these example datasets - this interactive session has everything you need to start solving these questions using SQL.

In [1]:
# Importing packages:
import pandas as pd
import os
import sqlalchemy

# Connecting to MySQL database:
host = os.environ.get('mysql_host')
user = os.environ.get('mysql_user')
password = os.environ.get('mysql_password')
engine = sqlalchemy.create_engine(f'mysql+pymysql://{user}:{password}@{host}/dannys_diner')

Use the following code to create the schema and the tables for the exercises:

In [ ]:
"""
CREATE SCHEMA IF NOT EXISTS dannys_diner;

CREATE TABLE dannys_diner.sales (
  customer_id VARCHAR(1),
  order_date DATE,
  product_id INTEGER
);

INSERT INTO dannys_diner.sales
  (customer_id, order_date, product_id)
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');


CREATE TABLE dannys_diner.menu (
  product_id INTEGER,
  product_name VARCHAR(5),
  price INTEGER
);

INSERT INTO dannys_diner.menu
  (product_id, product_name, price)
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');


CREATE TABLE dannys_diner.members (
  customer_id VARCHAR(1),
  join_date DATE
);

INSERT INTO dannys_diner.members
  (customer_id, join_date)
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
"""

## Case Study Questions
Each of the following case study questions can be answered using a single SQL statement:

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [22]:
# Answer 1
pd.read_sql_query(
    """
    SELECT
      s.customer_id,
      SUM(m.price) AS total_spent
    FROM sales s
    INNER JOIN menu m
      ON s.product_id=m.product_id
    GROUP BY s.customer_id
    ORDER BY total_spent DESC;
    """,
    engine
)

,customer_id,total_spent
0,A,76.0
1,B,74.0
2,C,36.0


In [26]:
# Answer 2
pd.read_sql_query(
    """
    SELECT
      customer_id,
      COUNT(DISTINCT order_date) AS visits_count
    FROM sales
    GROUP BY customer_id
    ORDER BY visits_count DESC;
    """,
    engine
)

,customer_id,visits_count
0,B,6
1,A,4
2,C,2


In [30]:
# Answer 3
pd.read_sql_query(
    """
    WITH cte_orders_ranks AS (
        SELECT
          s.*,
          m.product_name,
          RANK() OVER (
            PARTITION BY customer_id ORDER BY order_date
          ) AS date_rank
        FROM sales s
        INNER JOIN menu m
          ON s.product_id=m.product_id
    )
    SELECT DISTINCT
      customer_id,
      product_name
    FROM cte_orders_ranks
    WHERE date_rank=1;
    """,
    engine
)

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry
3,C,ramen


In [37]:
# Answer 4
pd.read_sql_query(
    """
    SELECT
      m.product_name,
      COUNT(*) AS product_count
    FROM sales s
    INNER JOIN menu m
      ON s.product_id=m.product_id
    GROUP BY product_name
    ORDER BY product_count DESC
    LIMIT 1;
    """,
    engine
)

,product_name,product_count
0,ramen,8


In [63]:
# Answer 5
pd.read_sql_query(
    """
    WITH cte_count AS (
    SELECT
      s.customer_id,
      m.product_name,
      COUNT(*) AS product_count
    FROM sales s
    INNER JOIN menu m
      ON s.product_id=m.product_id
    GROUP BY s.customer_id, m.product_name
    ),

    cte_ranks AS (
      SELECT
        *,
        RANK() OVER (
          PARTITION BY customer_id ORDER BY product_count DESC
        ) AS product_rank
      FROM cte_count
    )
    SELECT
      *
    FROM cte_ranks
    WHERE product_rank=1;
    """,
    engine
)

,customer_id,product_name,product_count,product_rank
0,A,ramen,3,1
1,B,curry,2,1
2,B,sushi,2,1
3,B,ramen,2,1
4,C,ramen,3,1


In [14]:
# Answer 6
pd.read_sql_query(
    """
    WITH cte_ranks AS (
        SELECT 
          s.*,
          m2.product_name,
          RANK() OVER (
            PARTITION BY customer_id ORDER BY order_date
          ) AS order_rank
        FROM sales s
        INNER JOIN members m1
          ON s.customer_id=m1.customer_id
          AND s.order_date>=m1.join_date
        INNER JOIN menu m2
          ON s.product_id=m2.product_id
    )
    SELECT
      customer_id,
      product_name
    FROM cte_ranks
    WHERE order_rank=1;
    """,
    engine
)

,customer_id,product_name
0,A,curry
1,B,sushi


In [21]:
# Answer 7
pd.read_sql_query(
    """
    WITH cte_ranks_before_members AS (
        SELECT
          s.*,
          m2.product_name,
          RANK() OVER (
            PARTITION BY customer_id ORDER BY order_date DESC
          ) AS order_rank
        FROM sales s
        INNER JOIN members m1
          ON s.customer_id=m1.customer_id
          AND s.order_date<m1.join_date
        INNER JOIN menu m2
          ON s.product_id=m2.product_id
    )
    SELECT
      customer_id,
    FROM cte_ranks_before_members
    WHERE order_rank=1;
    """,
    engine
)

,customer_id,order_date,product_id,product_name,order_rank
0,A,2021-01-01,1,sushi,1
1,A,2021-01-01,2,curry,1
2,B,2021-01-04,1,sushi,1


In [31]:
# Answer 8
pd.read_sql_query(
    """
    SELECT
      s.customer_id,
      COUNT(*) AS total_items,
      SUM(m2.price) AS total_spent
    FROM sales s
    INNER JOIN members m1
      ON s.customer_id=m1.customer_id
      AND s.order_date<m1.join_date
    INNER JOIN menu m2
      ON s.product_id=m2.product_id
    GROUP BY s.customer_id
    ORDER BY customer_id, total_spent DESC;
    """,
    engine
)

,customer_id,total_items,total_spent
0,A,2,25.0
1,B,3,40.0


In [37]:
# Answer 9
pd.read_sql_query(
    """
    WITH cte_points AS (
        SELECT
          *,
          CASE WHEN product_name='sushi' THEN price*20 ELSE price*10 END AS points
        FROM menu
    )
    SELECT 
      s.customer_id,
      SUM(p.points) AS total_points
    FROM sales s
    INNER JOIN cte_points p
      ON s.product_id=p.product_id
    GROUP BY s.customer_id
    ORDER BY customer_id;
    """,
    engine
)

,customer_id,total_points
0,A,860.0
1,B,940.0
2,C,360.0


10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [20]:
# Answer 10
pd.read_sql_query(
    """
    WITH cte_points AS (
        SELECT 
          s.*,
          m1.product_name,
          m1.price,
          m2.join_date,
          CASE
            WHEN s.order_date>=m2.join_date AND s.order_date<DATE_ADD(m2.join_date, INTERVAL 1 WEEK) THEN price * 20
            ELSE price * 10
          END AS points
        FROM sales s
        INNER JOIN menu m1
          ON s.product_id=m1.product_id
        LEFT JOIN members m2
          ON s.customer_id=m2.customer_id
    )
    SELECT
      customer_id,
      SUM(points) AS total_points
    FROM cte_points
    WHERE
      order_date<='2021-01-31'
      AND join_date IS NOT NULL
    GROUP BY customer_id
    ORDER BY customer_id;
    """,
    engine
)

,customer_id,total_points
0,A,1270.0
1,B,720.0


# References

[Data With Danny 8 Weeks Challenge - Case Study #1](https://8weeksqlchallenge.com/case-study-1/)